In [37]:
import pandas as pd
import numpy as np

In [50]:
df = pd.read_csv('../features/player_features.csv')
df.head(5)

,name,hand,ht,ioc,age,tourney_date,total_matches,wins,losses,win_rate,avg_minutes,avg_sets_won,avg_sets_lost,avg_opponent_age,avg_opponent_ht
0,Kim Warwick,R,183.0,AUS,19.7,1972-01-17,1.0,1.0,0.0,1.0,140.000000,2.000000,1.0,34.500000,173.0
1,Ion Tiriac,R,185.0,ROU,35.7,1975-02-24,1.0,1.0,0.0,1.0,25.000000,2.000000,0.0,0.000000,0.0
2,Roger Taylor,L,183.0,GBR,33.3,1975-02-24,1.0,1.0,0.0,1.0,135.000000,2.000000,1.0,22.800000,178.0
3,Christopher Rungkat,R,175.0,INA,26.1,2016-03-04,2.0,2.0,0.0,1.0,87.000000,3.000000,0.0,20.650000,175.0
4,Tucker Vorster,R,196.0,RSA,27.8,2016-07-15,3.0,3.0,0.0,1.0,95.333333,2.333333,0.0,20.233333,189.0


In [51]:
df = pd.read_csv('../data/atp_matches_all.csv')
df.head(5)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1968-2029,Dublin,Grass,32.0,A,19680708,270,112411,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1968-2029,Dublin,Grass,32.0,A,19680708,271,126914,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1968-2029,Dublin,Grass,32.0,A,19680708,272,209523,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1968-2029,Dublin,Grass,32.0,A,19680708,273,100084,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968-2029,Dublin,Grass,32.0,A,19680708,274,100132,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
X = ["tourney_date",
    "winner_hand", "winner_ht", "winner_ioc", "winner_age", "winner_name", "winner_seed",
    "loser_hand", "loser_ht", "loser_ioc", "loser_age", "loser_name", "loser_seed"
]

y = [
    "score", "minutes"
    ]

filtered_df = df[X + y]

filtered_df.shape, df.shape


((194996, 14), (194996, 49))

In [30]:
# Берем те строки, где есть значения в score и minutes
filtered_df = filtered_df.dropna(subset=["minutes"])
filtered_df = filtered_df.dropna(subset=["score"])

# Заполни ["loser_seed", "winner_seed"] nan на 0 и Преобразуем в int
filtered_df["loser_seed"] = filtered_df["loser_seed"].fillna(0).astype(int)
filtered_df["winner_seed"] = filtered_df["winner_seed"].fillna(0).astype(int)


filtered_df.shape

(95105, 14)

In [46]:
# filtered_df.info()

# len(list(set(list(filtered_df.winner_name.unique()) + list(filtered_df.loser_name.unique()))))

len(list(filtered_df.winner_name.unique()) + list(filtered_df.loser_name.unique()))

4161

In [32]:
import re

def calculate_set_score(score_string):
    if not isinstance(score_string, str):
        return ''
    
    sets = score_string.strip().split()
    player1_wins = 0
    player2_wins = 0
    
    for set_score in sets:
        # Убираем тай-брейк в скобках, если есть
        clean_score = re.sub(r'\(.*?\)', '', set_score)
        try:
            p1, p2 = map(int, clean_score.split('-'))
            if p1 > p2:
                player1_wins += 1
            else:
                player2_wins += 1
        except ValueError:
            # Невалидный формат — пропускаем
            continue

    return f"{player1_wins}:{player2_wins}"


filtered_df['score'] = filtered_df['score'].apply(calculate_set_score)

In [54]:
filtered_df.head(5)
# 0:3

# посчитай мне количество значений  для каждого score
filtered_df['score'].value_counts()
# filtered_df[filtered_df['score'] == '0:3']

,winner_hand,winner_ht,winner_ioc,winner_age,winner_name,winner_seed,loser_hand,loser_ht,loser_ioc,loser_age,loser_name,loser_seed,score,minutes
145561,L,188.0,ESP,23.1,Fernando Verdasco,0,R,185.0,FRA,25.0,Paul Henri Mathieu,0,0:3,180.0


In [10]:
filtered_df.head(5)[['score', 'minutes']]['minutes'][17897]

filtered_df['score'].unique()

array(['2:1', '2:0', '3:0', '1:1', '3:2', '3:1', '1:0', '1:2', '0:2',
       '0:1', '0:0', '2:2', '2:3', '0:3'], dtype=object)

In [36]:
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import joblib

# Загрузка данных
df = filtered_df

# Фильтрация
df = df[['winner_rank', 'loser_rank', 'winner_age', 'loser_age', 'surface', 'tourney_level', 'score', 'minutes']]
df = df[df['score'].notna() & df['minutes'].notna()]

# Целевая переменная
df['score_class'] = df['score']

# Обрабатываем категориальные как строки
df['surface'] = df['surface'].astype(str)
df['tourney_level'] = df['tourney_level'].astype(str)

# Признаки
features = ['winner_rank', 'loser_rank', 'winner_age', 'loser_age', 'surface', 'tourney_level']
cat_features = ['surface', 'tourney_level']

X = df[features]
y_class = df['score_class']
y_reg = df['minutes']

# Классификация
clf = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=3,
    cat_features=cat_features,
    verbose=0
)
clf.fit(X, y_class)

# Регрессия
reg = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.1,
    depth=3,
    cat_features=cat_features,
    verbose=0
)
reg.fit(X, y_reg)

# Сохраняем
joblib.dump(clf, 'model/score_predictor_catboost.pkl')
joblib.dump(reg, 'model/minutes_predictor_catboost.pkl')



['model/minutes_predictor_catboost.pkl']

In [44]:
filtered_df['score'].unique()

# посчитай сколько раз встречается каждая строка
filtered_df['score'].value_counts()

score
2:0    50928
2:1    27460
3:0     7557
3:1     4909
3:2     3038
1:0      575
1:1      326
0:1      111
1:2       70
0:0       62
2:2       33
0:2       28
2:3        7
0:3        1
Name: count, dtype: int64

In [ ]:
df

In [37]:
import pandas as pd
import joblib

# Загрузка данных
df = filtered_df

# Глобальные средние значения для признаков
mean_values = {
    'winner_rank': df['winner_rank'].mean(),
    'loser_rank': df['loser_rank'].mean(),
    'winner_age': df['winner_age'].mean(),
    'loser_age': df['loser_age'].mean(),
    'surface': df['surface'].mode()[0],
    'tourney_level': df['tourney_level'].mode()[0]
}

def predict_score(player1, player2):
    # Загрузка моделей
    clf_model = joblib.load('model/score_predictor_catboost.pkl')
    reg_model = joblib.load('model/minutes_predictor_catboost.pkl')

    # Получаем данные о последних матчах игроков
    p1 = df[df['winner_name'] == player1].iloc[-1:]
    p2 = df[df['loser_name'] == player2].iloc[-1:]

    # Если данных нет, используем средние значения
    if p1.empty or p2.empty:
        print("Нет данных о таком матче, используя средние значения для предсказания.")
        
        features = {
            'winner_rank': mean_values['winner_rank'],
            'loser_rank': mean_values['loser_rank'],
            'winner_age': mean_values['winner_age'],
            'loser_age': mean_values['loser_age'],
            'surface': mean_values['surface'],
            'tourney_level': mean_values['tourney_level']
        }
    else:
        features = {
            'winner_rank': p1['winner_rank'].values[0],
            'loser_rank': p2['loser_rank'].values[0],
            'winner_age': p1['winner_age'].values[0],
            'loser_age': p2['loser_age'].values[0],
            'surface': p1['surface'].values[0],
            'tourney_level': p1['tourney_level'].values[0]
        }

    input_df = pd.DataFrame([features])

    # Предсказания
    predicted_score = clf_model.predict(input_df)[0]
    predicted_minutes = reg_model.predict(input_df)[0]

    return f"Предсказанный счёт: {predicted_score[0]}, длительность матча: {round(predicted_minutes)} мин."


In [38]:
print(predict_score("Novak Djokovic", "Carlos Alcaraz"))
# ➜ "Предсказанный счёт: 3:1, длительность матча: 142 мин."

Предсказанный счёт: 2:0, длительность матча: 113 мин.


In [39]:
len(filtered_df.winner_name.unique())

# напиши цикл где будут браться два случйаных игрока # и предсказывать их матч
import random


def random_match_prediction(df):
    players_1 = df.loser_name.unique()
    players_2 = df.winner_name.unique()
    players = players_1.tolist() + players_2.tolist()
    player1 = random.choice(players)
    player2 = random.choice(players)
    while player1 == player2:
        player2 = random.choice(players)
    return predict_score(player1, player2), (player1, player2)

for i in range(10):
    print(random_match_prediction(filtered_df))

('Предсказанный счёт: 2:0, длительность матча: 100 мин.', ('Francisco Clavet', 'Somdev Devvarman'))
Нет данных о таком матче, используя средние значения для предсказания.
('Предсказанный счёт: 2:0, длительность матча: 94 мин.', ('Kim Tiilikainen', 'Felix Auger Aliassime'))
('Предсказанный счёт: 2:0, длительность матча: 100 мин.', ('Daniel Gimeno Traver', 'Bernard Tomic'))
('Предсказанный счёт: 2:0, длительность матча: 103 мин.', ('Agustin Calleri', 'Antoine Hoang'))
Нет данных о таком матче, используя средние значения для предсказания.
('Предсказанный счёт: 2:0, длительность матча: 94 мин.', ('Ainius Sabaliauskas', 'Razvan Sabau'))
('Предсказанный счёт: 2:0, длительность матча: 93 мин.', ('Alistair Hunt', 'Tommy Ho'))
Нет данных о таком матче, используя средние значения для предсказания.
('Предсказанный счёт: 2:0, длительность матча: 94 мин.', ('Sebastian Soresini', 'John Bryan Decasa Otico'))
('Предсказанный счёт: 3:0, длительность матча: 139 мин.', ('Omar Jasika', 'Dane Sweeny'))
('П